In [1]:
import pyodbc
import pandas as pd
from boxsdk import JWTAuth, Client
from datetime import datetime, timezone
import os
import logging
from logging.handlers import TimedRotatingFileHandler

conn_str = (
    r'Driver={ODBC Driver 17 for SQL Server};'
    r'Server=dca1-up-dls;'
    r'Database=IACOE;'
    r'Trusted_Connection=yes;'
    )

cnxn = pyodbc.connect(conn_str)

cursor = cnxn.cursor()

query = """SELECT * FROM TaxML_Validation_Data"""

data = pd.read_sql(query,cnxn)
#cnxn.commit()
#cnxn.close()


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503934 entries, 0 to 1503933
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   CICD Run Date                        1468441 non-null  datetime64[ns]
 1   Date (BOT Sent Details to COE Team)  1468441 non-null  datetime64[ns]
 2   UniqueUUID                           1503934 non-null  object        
 3   store_uuid                           1503934 non-null  object        
 4   item_uuid                            1503934 non-null  object        
 5   Item                                 1503920 non-null  object        
 6   Description                          930243 non-null   object        
 7   establishment_type                   1503930 non-null  object        
 8   ML CAT Name                          1503472 non-null  object        
 9   ML Integer                           1503806 non-null  ob

In [3]:
master_mapping_data = pd.read_csv("./MASTER_Tax_Category_Mapping.csv")
set_cicd_categories_aftermapping = set(data['Agent CAT Name'].unique())
set_master_mapping_data_categories = set(master_mapping_data['CAT_Name'].unique())
dif_categories = list(set_cicd_categories_aftermapping.difference(set_master_mapping_data_categories))
len(dif_categories)

21

In [4]:
master_mapping_data = pd.read_csv("./MASTER_Tax_Category_Mapping.csv")
set_cicd_categories_aftermapping = set(data['ML CAT Name'].unique())
set_master_mapping_data_categories = set(master_mapping_data['CAT_Name'].unique())
dif_categories = list(set_cicd_categories_aftermapping.difference(set_master_mapping_data_categories))
len(dif_categories)

1

In [5]:
dif_categories

[None]

In [6]:
mapping_data=pd.read_csv('./mapping_latest.csv')
cat_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated CAT Name']))
int_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated Integer']))
data["ML CAT Name"] = data["ML CAT Name"].map(cat_dict)
data["ML Integer"] = data["ML CAT Name"].map(int_dict)

In [7]:
mapping_data

,Agent CAT Name,Updated CAT Name,Updated Integer
0,CAT_ALCOHOL,"CAT_ALCOHOL,TEMP_COLD","109,3"
1,CAT_CANDY,"CAT_CANDY,TEMP_COLD","108,3"
2,"CAT_CANDY, TRAIT_FLOUR","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD","108,61,3"
3,"CAT_ENERGY_DRINK, CONTAINER_BOTTLED","CAT_ENERGY_DRINK,CONTAINER_BOTTLED,TEMP_COLD","113,11,3"
4,CAT_ICE,"CAT_ICE,TEMP_COLD","118,3"
...,...,...,...
115,Toothpaste,CAT_TOOTHBRUSH,549
116,Toys and Games,CAT_TPP_TOYS,822
117,Vitamins and Supplements,CAT_SUPPLEMENTS,542
118,"CAT_LIQUOR,TEMP_HEATED",CAT_LIQUOR,535


In [8]:
cnxn = pyodbc.connect(conn_str)

cursor = cnxn.cursor()

In [88]:
for index, row in mapping_data.iterrows():
    #uuid = row["UniqueUUID"]
    updatedcatname = row["Updated CAT Name"]
    updatedinteger = row["Updated Integer"]
    agentcatname = row["Agent CAT Name"]
    query1 = f""" UPDATE TaxML_Validation_Data SET [ML CAT Name] = '{updatedcatname}', [ML Integer] = '{updatedinteger}'
                Where [ML CAT Name] = '{agentcatname}' """
    cursor.execute(query1)
   # print('1')
    cnxn.commit()
#cnxn.close()

In [9]:
for index, row in mapping_data.iterrows():
    #uuid = row["UniqueUUID"]
    updatedcatname = row["Updated CAT Name"]
    updatedinteger = row["Updated Integer"]
    agentcatname = row["Agent CAT Name"]
    #print(updatedcatname,updatedinteger,agentcatname)
    query2 = f""" UPDATE TaxML_Validation_Data SET [Agent CAT Name] = '{updatedcatname}', [Agent Integer] = '{updatedinteger}'
                Where [Agent CAT Name] = '{agentcatname}' """
    cursor.execute(query2)
    cnxn.commit()
#cnxn.close()

In [38]:
for index, row in mapping_data.iterrows():
    uuid = row["UniqueUUID"]
    updatedcatname = row["ML CAT Name"]
    updatedinetger = row["ML Integer"]
    updatedconf = row["ML Confidence Score"]
    query1 = f""" UPDATE TaxML_Validation_Data SET [ML CAT Name] = '{updatedcatname}', [ML Integer] = '{updatedinetger}', [ML Confidence Score] = '{updatedconf}'
                Where [Agent CAT Name] = '{uuid}' """
    cursor.execute(query1)
    cnxn.commit()

In [2]:
agentcatname = '-2146826265'
updatedcatname = ' '
updatedinetger = ' '
query1 = f""" UPDATE TaxML_Validation_Data SET [Agent CAT Name] = NULL, [Agent Integer] =  NULL
            Where [Agent CAT Name] = '{updatedinetger}' """
cursor.execute(query1)
cnxn.commit()

In [66]:
#agentcatname = ' '
updatedinetger = ' '
query1 = f""" UPDATE TaxML_Validation_Data SET  [Agent Integer] = NULL
            Where [Agent CAT Name] is null """
cursor.execute(query1)
cnxn.commit()

In [43]:
#agentcatname = '746'
updatedcatname = '-2146826265'
#updatedinetger = '746'
query1 = f""" UPDATE TaxML_Validation_Data SET [ML Confidence Score] = '{updatedcatname}'
            Where [ML Confidence Score] is null """
cursor.execute(query1)
cnxn.commit()

In [75]:
#for index, row in data.iterrows():
query3 = f"""UPDATE TaxML_Validation_Data 
            SET [Accuracy] = 0 """
cursor.execute(query3)
cnxn.commit()

In [79]:
#for index, row in data.iterrows():
query3 = f"""UPDATE TaxML_Validation_Data 
            SET [Accuracy] = 1 
            Where [ML CAT Name] = [Agent CAT Name]"""
cursor.execute(query3)
cnxn.commit()

In [85]:
query4 = f"""SELECT *, ROUND(CAST(e.predicted_true_count AS DECIMAL(16,2)) / e.total_volume,2) AS accuracy
                FROM (SELECT
                    [Establishment_type],[ML CAT Name],[Agent CAT Name], [Accuracy],
                    COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL THEN [Agent CAT Name] END) AS total_volume,
                    COUNT(CASE WHEN [ML CAT Name]=[Agent CAT Name] THEN [Agent CAT Name] END) AS predicted_true_count
                FROM TaxML_Validation_Data
                GROUP BY [Establishment_type],[ML CAT Name],[Agent CAT Name], [Accuracy]) AS e
                WHERE e.predicted_true_count >= 1
                ORDER BY e.predicted_true_count DESC """
data_byET = pd.read_sql(query4,cnxn)
#cnxn.commit()

In [86]:
data_byET.to_csv("data_byET.csv")

In [25]:
data_updated = pd.read_sql(query,cnxn)
#cnxn.close()
data_updated.head()

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,ML CAT Name,ML Integer,ML Confidence Score,Agent CAT Name,Agent Integer,Accuracy,New ML Prediction,New ML Integer,New Accuracy,Module
0,2022-08-09 10:36:00,2022-06-30 11:11:00,000168cb-e79b-4299-af44-4f86e36a8464:c17291c5-...,000168cb-e79b-4299-af44-4f86e36a8464,c17291c5-83d2-5155-85d0-305e41695b95,Egg White & Veggie Omelet Bites,"Cage-free eggs mixed with spinach, tomatoes an...",DEFAULT,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",0.9,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",True,None,None,None,Restaurant
1,2022-05-26 19:54:00,2022-05-19 19:08:00,0001a851-7ca3-484f-b3e6-75899dd18207:1b0bb2ba-...,0001a851-7ca3-484f-b3e6-75899dd18207,1b0bb2ba-99c3-574f-b203-e9b3388d5184,Ensenada Chicken Platter,One or two ancho-grilled chicken breasts house...,DEFAULT,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",1.0,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",True,None,None,None,Restaurants
2,2022-05-26 19:54:00,2022-05-19 19:08:00,0001a851-7ca3-484f-b3e6-75899dd18207:2f7fc8d6-...,0001a851-7ca3-484f-b3e6-75899dd18207,2f7fc8d6-8167-5521-b3ee-7e19281e6d60,Get Saucy,Our sauce secret is out Creamy Ranch Dressing ...,DEFAULT,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",0.7,CAT_PREPACKAGED_FOOD_CONDIMENTS,740,True,None,None,None,Restaurants
3,2022-05-26 19:54:00,2022-05-19 19:08:00,0001a851-7ca3-484f-b3e6-75899dd18207:3bc15321-...,0001a851-7ca3-484f-b3e6-75899dd18207,3bc15321-d86f-51a9-a12b-438a8c69a591,Donatos 10 Pizza,10 Donatos famous crispy thin crust loaded Edg...,DEFAULT,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",0.6,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",True,None,None,None,Restaurants
4,2022-05-26 19:54:00,2022-05-19 19:08:00,0001a851-7ca3-484f-b3e6-75899dd18207:44c840cc-...,0001a851-7ca3-484f-b3e6-75899dd18207,44c840cc-3544-55b6-969c-56902c07899a,Donatos 7 Pizza,Loaded Edge to Edge with our heritage recipe P...,DEFAULT,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",0.6,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",True,None,None,None,Restaurants


In [7]:
agentcatname_list = list(mapping_data['Agent CAT Name'])
#data_updated.loc[data_updated['Agent CAT Name'] == "Antidiarrheals"]
data_updated.loc[data_updated['Agent CAT Name'].isin(agentcatname_list)]

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,ML CAT Name,ML Integer,ML Confidence Score,Agent CAT Name,Agent Integer,Accuracy,New ML Prediction,New ML Integer,New Accuracy,Module


In [11]:
agentcatname_list = list(mapping_data['Agent CAT Name'])
#data_updated.loc[data_updated['Agent CAT Name'] == "Antidiarrheals"]
data_updated.loc[data_updated['ML CAT Name'].isin(agentcatname_list)]

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,ML CAT Name,ML Integer,ML Confidence Score,Agent CAT Name,Agent Integer,Accuracy,New ML Prediction,New ML Integer,New Accuracy,Module


In [21]:
count = data_updated.nunique()
count

CICD Run Date                              227
Date (BOT Sent Details to COE Team)       2908
UniqueUUID                             1134464
store_uuid                               31308
item_uuid                              1104264
Item                                    354568
Description                             137909
establishment_type                          12
ML CAT Name                                260
ML Integer                                 277
ML Confidence Score                       1197
Agent CAT Name                             195
Agent Integer                              220
Accuracy                                     2
New ML Prediction                            1
New ML Integer                               1
New Accuracy                                 1
Module                                       4
dtype: int64

In [23]:
data['Accuracy'].value_counts()

True     929429
False    205035
Name: Accuracy, dtype: int64

In [26]:
data_updated['Accuracy'].value_counts()

True     1131558
False       2906
Name: Accuracy, dtype: int64